In [3]:
import pandas as pd
import json
import os
import sys

# パス設定
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# 必要なモジュール
from src.bridge.excel_parser import parse_excel_spec
from src.schema.definitions import OutlookConfig
from src.adapter.outlook import OutlookAdapter
from src.engine.core import GenericEtlEngine
import src.catalog.handlers.basic
import src.catalog.handlers.document
import src.catalog.handlers.archive

# =================================================================
# Step 1: Excelファイルの準備
# =================================================================
def create_mock_excel(file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Sheet 1: Settings
    df_settings = pd.DataFrame([
        ["Job Name", "Excel_Driven_Bot_V2"],
        ["Domain", "Accounting"],
        ["Keywords", "請求書, Invoice"],
        ["Destination", "./data/excel_output"]
    ])
    
    # Sheet 2: Rules
    df_rules = pd.DataFrame([
        [".pdf",  "pdf_to_text_ocr", '{"lang": "jpn"}'],
        [".xlsx", "save_only",       ''],
        [".zip",  "unzip_file",      '{"mode": "manual"}'] 
    ], columns=["Extension", "Processor ID", "Parameters"])

    with pd.ExcelWriter(file_path) as writer:
        df_settings.to_excel(writer, sheet_name="Settings", header=False, index=False)
        df_rules.to_excel(writer, sheet_name="Rules", index=False)
    
    print(f"1️⃣  [Human] Excel仕様書を作成しました: {file_path}")

# =================================================================
# Step 2: Bridgeの実行 (★ここを修正しました)
# =================================================================
def run_bridge_compile(excel_path, json_out_path):
    print(f"2️⃣  [Bridge] コンパイル開始...")
    try:
        config = parse_excel_spec(excel_path)
        
        os.makedirs(os.path.dirname(json_out_path), exist_ok=True)
        with open(json_out_path, "w", encoding="utf-8") as f:
            # ▼▼▼ 修正: バージョン互換対応 ▼▼▼
            if hasattr(config, "model_dump_json"):
                # Pydantic v2
                f.write(config.model_dump_json(indent=2))
            else:
                # Pydantic v1
                f.write(config.json(indent=2))
            # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
            
        print(f"    ✅ JSON生成成功: {json_out_path}")
        return True
    except Exception as e:
        print(f"    ❌ コンパイル失敗: {e}")
        return False

# =================================================================
# Step 3: Engineの実行
# =================================================================
def run_engine(json_path):
    print(f"3️⃣  [Engine] ロボット起動...")
    
    if not os.path.exists(json_path):
        print("❌ JSONファイルがありません。Step 2を確認してください。")
        return

    with open(json_path, "r", encoding="utf-8") as f:
        config_data = json.load(f)
        config = OutlookConfig(**config_data)

    adapter = OutlookAdapter()
    engine = GenericEtlEngine(config, adapter)
    engine.run()

# =================================================================
# 🔥 メイン実行ブロック
# =================================================================
EXCEL_PATH = "specs/accounting/invoice_bot_v2.xlsx"
JSON_PATH = "configs/accounting/invoice_bot_v2.json"

create_mock_excel(EXCEL_PATH)

if run_bridge_compile(EXCEL_PATH, JSON_PATH):
    print("-" * 50)
    run_engine(JSON_PATH)

1️⃣  [Human] Excel仕様書を作成しました: specs/accounting/invoice_bot_v2.xlsx
2️⃣  [Bridge] コンパイル開始...
📖 Excel仕様書を解析中...: specs/accounting/invoice_bot_v2.xlsx
✅ 解析完了: Job='Excel_Driven_Bot_V2' / Domain='Accounting' / Rules=3件
    ✅ JSON生成成功: configs/accounting/invoice_bot_v2.json
--------------------------------------------------
3️⃣  [Engine] ロボット起動...
🚀 Engine Start: Excel_Driven_Bot_V2 (v2.0)
>> [Adapter] Outlookに接続しました
>> [Adapter] 検索 '請求書': 3 件ヒット
  - メール発見: Microsoft 請求書 G128449371 の準備ができました
    ⚙️ Executing ProcessorType.PDF_OCR for G128449371.pdf...
    ⚠️ v1互換モードで実行します: logic_pdf_ocr() takes 2 positional arguments but 3 were given
    [OCR ] G128449371.pdf をテキスト化しました -> G128449371.pdf.txt
  - メール発見: Microsoft 請求書 G133796987 の準備ができました
    ⚙️ Executing ProcessorType.PDF_OCR for G133796987.pdf...
    ⚠️ v1互換モードで実行します: logic_pdf_ocr() takes 2 positional arguments but 3 were given
    [OCR ] G133796987.pdf をテキスト化しました -> G133796987.pdf.txt
  - メール発見: テスト用請求書
    ⚙️ Executing ProcessorType.SAVE